In [ ]:
import os
import pandas as pd
from pathlib import Path
import json
import re
from stereomapper.domain.chemistry.analysis import StereoAnalyser
from stereomapper.domain.chemistry import ChemistryUtils
from stereomapper.domain.chemistry import OpenBabelOperations
import sqlite3
import subprocess

In [ ]:
# run stereomapper as a subprocess
enantiomers = Path.home('enantiomer_benchmark_data')
results = Path.home('enantiomer_benchmark_results.sqlite') 
cache_path = Path.home('enantiomer_benchmark_cache.sqlite')

cmd =[
    "stereomapper",
    "run",
    "-d", enantiomers.as_posix(),
    "-o", results.as_posix(),
    "-p", cache_path.as_posix(),
    "--fresh-cache"
]

subprocess.run(cmd) 

INFO    Logging initialised. File: /media/JACK/stereomapper_logs/stereomapper_20251104_101611.log
Pipeline: 100%|██████████████████████████████████████████████████████████| , Complete! [00:07<00:00]


✅ Pipeline completed in 7.5s
📦 Inputs attempted: 2,504 (skipped 0)
📊 Successes: 2,500 | Failures: 4
🔗 Inchikey groups — processed 931, skipped 0, failed 0
🧮 Relationship rows: 3,618
🧾 Unique inchikeys observed: 931
💾 Cache hit rate: 0.0%


Pipeline: 100%|██████████████████████████████████████████████████████████| , Complete! [00:08<00:00]


CompletedProcess(args=['stereomapper', 'run', '-d', '/home/jackmcgoldrick/drive/Recon4IMD/WP5_Reconstruction/T5.1_ReconXKG/data/stereomapper/control_sets/stereoisomer_control/enantiomer_benchmark_data', '-o', '/home/jackmcgoldrick/drive/Recon4IMD/WP5_Reconstruction/T5.1_ReconXKG/results/stereomapper/project_results/benchmarking/041125_enantiomer_benchmark.sqlite', '-p', '/home/jackmcgoldrick/drive/Recon4IMD/WP5_Reconstruction/T5.1_ReconXKG/results/stereomapper/project_results/benchmarking/cache/041125_enantiomer_benchmark.sqlite', '--fresh-cache'], returncode=0)

In [44]:
conn = sqlite3.connect(results)
merged_df_q = """ 
SELECT * from relationships;
"""
df_merged = pd.read_sql_query(merged_df_q, conn)
df_merged

,cluster_a,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag
0,1,2,"[""chebi:CHEBI:17521""]","[""chebi:CHEBI:36124""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
1,3,4,"[""chebi:CHEBI:137507""]","[""chebi:CHEBI:137513""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
2,5,6,"[""chebi:CHEBI:134198""]","[""chebi:CHEBI:134199""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3,7,8,"[""chebi:CHEBI:28651""]","[""chebi:CHEBI:27702""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
4,9,10,"[""chebi:CHEBI:145480""]","[""chebi:CHEBI:145483""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
...,...,...,...,...,...,...,...,...,...,...,...
3613,2456,2458,"[""chebi:CHEBI:133313""]","[""chebi:CHEBI:133312""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3614,2457,2458,"[""chebi:CHEBI:133311""]","[""chebi:CHEBI:133312""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3615,2459,2460,"[""chebi:CHEBI:233960""]","[""chebi:CHEBI:233959""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3616,2461,2462,"[""chebi:CHEBI:142550""]","[""chebi:CHEBI:9399""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0


In [52]:
expanded_keys = set()
for _, row in df_merged.iterrows():
    if not row["cluster_a_members"] or not row["cluster_b_members"]:
        continue
    m1s = [norm_chebi(x) for x in json.loads(row["cluster_a_members"])]
    m2s = [norm_chebi(x) for x in json.loads(row["cluster_b_members"])]
    for a in m1s:
        for b in m2s:
            if a == b:
                continue
            expanded_keys.add(tuple(sorted((a, b))))

# --- Overlap ---
tp = control_keys & expanded_keys
fp = expanded_keys - control_keys
fn = control_keys - expanded_keys

print(f"True Positives: {len(tp)}")
print(f"False Positives: {len(fp)}")
print(f"False Negatives: {len(fn)}")


True Positives: 1243
False Positives: 2494
False Negatives: 119


Not a true representation of the results, contains 2384 false positives majority of which are off target relationships. Therefore, the dataset needs to be remapped back onto the original pairs stored in the control dataset.

In [53]:
valid_pairs = control_keys

df_pred_filtered = []
for _, row in df_merged.iterrows():
    members_1 = [norm_chebi(x) for x in json.loads(row["cluster_a_members"])]
    members_2 = [norm_chebi(x) for x in json.loads(row["cluster_b_members"])]
    for a in members_1:
        for b in members_2:
            pair = tuple(sorted((a, b)))
            if pair in valid_pairs:
                df_pred_filtered.append({"id1": a, "id2": b, **row.to_dict()})

df_pred_filtered = pd.DataFrame(df_pred_filtered)

In [54]:
df_pred_filtered = df_pred_filtered.copy()
df_pred_filtered["pair_key"] = df_pred_filtered.apply(
    lambda r: tuple(sorted([norm_chebi(r["id1"]), norm_chebi(r["id2"])])), axis=1
)

In [55]:
df_pred_filtered

,id1,id2,cluster_a,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag,pair_key
0,CHEBI_17521,CHEBI_36124,1,2,"[""chebi:CHEBI:17521""]","[""chebi:CHEBI:36124""]",1,1,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0,"(CHEBI_17521, CHEBI_36124)"
1,CHEBI_137507,CHEBI_137513,3,4,"[""chebi:CHEBI:137507""]","[""chebi:CHEBI:137513""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0,"(CHEBI_137507, CHEBI_137513)"
2,CHEBI_134198,CHEBI_134199,5,6,"[""chebi:CHEBI:134198""]","[""chebi:CHEBI:134199""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0,"(CHEBI_134198, CHEBI_134199)"
3,CHEBI_28651,CHEBI_27702,7,8,"[""chebi:CHEBI:28651""]","[""chebi:CHEBI:27702""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0,"(CHEBI_27702, CHEBI_28651)"
4,CHEBI_145480,CHEBI_145483,9,10,"[""chebi:CHEBI:145480""]","[""chebi:CHEBI:145483""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0,"(CHEBI_145480, CHEBI_145483)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,CHEBI_133315,CHEBI_133312,2455,2458,"[""chebi:CHEBI:133315""]","[""chebi:CHEBI:133312""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0,"(CHEBI_133312, CHEBI_133315)"
1239,CHEBI_133313,CHEBI_133311,2456,2457,"[""chebi:CHEBI:133313""]","[""chebi:CHEBI:133311""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0,"(CHEBI_133311, CHEBI_133313)"
1240,CHEBI_233960,CHEBI_233959,2459,2460,"[""chebi:CHEBI:233960""]","[""chebi:CHEBI:233959""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0,"(CHEBI_233959, CHEBI_233960)"
1241,CHEBI_142550,CHEBI_9399,2461,2462,"[""chebi:CHEBI:142550""]","[""chebi:CHEBI:9399""]",1,1,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0,"(CHEBI_142550, CHEBI_9399)"


In [56]:
df_eval = pd.merge(
    dfc_dedup_wcs_excluded, df_pred_filtered,
    on="pair_key",
    how="left",
    suffixes=("_true", "_pred")
)

In [57]:
df_eval

,id1_true,id2_true,label,molfile1,molfile2,id1_c,id2_c,pair_key,id1_pred,id2_pred,...,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag
0,CHEBI_43796,CHEBI_30314,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_43796,CHEBI_30314,"(CHEBI_30314, CHEBI_43796)",CHEBI_30314,CHEBI_43796,...,34.0,"[""chebi:CHEBI:30314""]","[""chebi:CHEBI:43796""]",1.0,1.0,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
1,CHEBI_15570,CHEBI_16977,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_15570,CHEBI_16977,"(CHEBI_15570, CHEBI_16977)",CHEBI_15570,CHEBI_16977,...,1555.0,"[""chebi:CHEBI:15570""]","[""chebi:CHEBI:16977""]",1.0,1.0,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
2,CHEBI_17561,CHEBI_16375,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_17561,CHEBI_16375,"(CHEBI_16375, CHEBI_17561)",CHEBI_17561,CHEBI_16375,...,2239.0,"[""chebi:CHEBI:17561"", ""chebi:CHEBI:35235""]","[""chebi:CHEBI:16375"", ""chebi:CHEBI:35236""]",2.0,2.0,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
3,CHEBI_17053,CHEBI_17364,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_17053,CHEBI_17364,"(CHEBI_17053, CHEBI_17364)",CHEBI_17053,CHEBI_17364,...,255.0,"[""chebi:CHEBI:17053""]","[""chebi:CHEBI:17364""]",1.0,1.0,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
4,CHEBI_29990,CHEBI_29991,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_29990,CHEBI_29991,"(CHEBI_29990, CHEBI_29991)",CHEBI_29991,CHEBI_29990,...,259.0,"[""chebi:CHEBI:29991""]","[""chebi:CHEBI:29990""]",1.0,1.0,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1247,CHEBI_233695,CHEBI_176808,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_233695,CHEBI_176808,"(CHEBI_176808, CHEBI_233695)",CHEBI_233695,CHEBI_176808,...,2181.0,"[""chebi:CHEBI:233695""]","[""chebi:CHEBI:176808""]",1.0,1.0,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
1248,CHEBI_233134,CHEBI_233952,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_233134,CHEBI_233952,"(CHEBI_233134, CHEBI_233952)",CHEBI_233952,CHEBI_233134,...,2225.0,"[""chebi:CHEBI:233952""]","[""chebi:CHEBI:233134""]",1.0,1.0,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
1249,CHEBI_233954,CHEBI_233133,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_233954,CHEBI_233133,"(CHEBI_233133, CHEBI_233954)",CHEBI_233954,CHEBI_233133,...,929.0,"[""chebi:CHEBI:233954""]","[""chebi:CHEBI:233133""]",1.0,1.0,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
1250,CHEBI_233959,CHEBI_233960,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_233959,CHEBI_233960,"(CHEBI_233959, CHEBI_233960)",CHEBI_233960,CHEBI_233959,...,2460.0,"[""chebi:CHEBI:233960""]","[""chebi:CHEBI:233959""]",1.0,1.0,Enantiomers,100.0,"{""confidence_bin"":""high""}",None,v1.0


Dataframe `df_eval` now contains the original pairs of identifiers with their predicitions from stereomapper. This dataframe will be used to accurately calculate the precision, recall and F1 score on the enantiomer control dataset.

In [58]:
# only rows where we have a prediction - removing FN, ensure to state in results section
df_with_pred = df_eval.dropna(subset=["classification"])

tp = ((df_with_pred["classification"] == df_with_pred["label"]).sum())
fp = ((df_with_pred["classification"] != df_with_pred["label"]).sum())
fn = df_eval["classification"].isna().sum()

print(f"TP: {tp}, FP: {fp}, FN: {fn}")

TP: 1180, FP: 63, FN: 9


119 false negatives indicates rows where structures either could not be processed (due to errors in their structure files or issues with the parser compatibility) or they were excluded due to presence of wildcard atoms. 

In [59]:
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1 score:  {f1:.3f}")


Precision: 0.949
Recall:    0.992
F1 score:  0.970


Great results, lets investigate the false positive results to see what went wrong and if the pipeline can be improved.

In [77]:
df_fp = df_with_pred[df_with_pred["classification"] != df_with_pred["label"]]
df_fp

,id1_true,id2_true,label,molfile1,molfile2,id1_c,id2_c,pair_key,id1_pred,id2_pred,...,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag
31,CHEBI_27372,CHEBI_27374,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_27372,CHEBI_27374,"(CHEBI_27372, CHEBI_27374)",CHEBI_27374,CHEBI_27372,...,233.0,"[""chebi:CHEBI:27374""]","[""chebi:CHEBI:27372""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
46,CHEBI_21397,CHEBI_21100,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_21397,CHEBI_21100,"(CHEBI_21100, CHEBI_21397)",CHEBI_21100,CHEBI_21397,...,417.0,"[""chebi:CHEBI:21100""]","[""chebi:CHEBI:21397""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
47,CHEBI_21101,CHEBI_21398,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_21101,CHEBI_21398,"(CHEBI_21101, CHEBI_21398)",CHEBI_21101,CHEBI_21398,...,409.0,"[""chebi:CHEBI:21101""]","[""chebi:CHEBI:21398""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
49,CHEBI_47537,CHEBI_16002,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_47537,CHEBI_16002,"(CHEBI_16002, CHEBI_47537)",CHEBI_16002,CHEBI_47537,...,404.0,"[""chebi:CHEBI:16002""]","[""chebi:CHEBI:47537""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
65,CHEBI_32922,CHEBI_35440,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_32922,CHEBI_35440,"(CHEBI_32922, CHEBI_35440)",CHEBI_32922,CHEBI_35440,...,1870.0,"[""chebi:CHEBI:32922""]","[""chebi:CHEBI:35440""]",1.0,1.0,Parent-child,79.0,"{""confidence_bin"":""medium""}",None,v1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,CHEBI_231950,CHEBI_231949,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_231950,CHEBI_231949,"(CHEBI_231949, CHEBI_231950)",CHEBI_231949,CHEBI_231950,...,2229.0,"[""chebi:CHEBI:231949""]","[""chebi:CHEBI:231950""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
1238,CHEBI_232288,CHEBI_76864,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_232288,CHEBI_76864,"(CHEBI_232288, CHEBI_76864)",CHEBI_76864,CHEBI_232288,...,782.0,"[""chebi:CHEBI:76864""]","[""chebi:CHEBI:232288""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
1240,CHEBI_232459,CHEBI_232353,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_232459,CHEBI_232353,"(CHEBI_232353, CHEBI_232459)",CHEBI_232353,CHEBI_232459,...,2299.0,"[""chebi:CHEBI:232353""]","[""chebi:CHEBI:232459""]",1.0,1.0,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
1241,CHEBI_232532,CHEBI_76557,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_232532,CHEBI_76557,"(CHEBI_232532, CHEBI_76557)",CHEBI_76557,CHEBI_232532,...,1710.0,"[""chebi:CHEBI:76557""]","[""chebi:CHEBI:232532""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0


In [70]:
df_fp['classification'].value_counts()

classification
Diastereomers    37
Parent-child     20
Unresolved        5
Unclassified      1
Name: count, dtype: int64

In [73]:
df_fp_dia = df_fp[df_fp['classification'] == "Diastereomers"]
df_fp_dia

,id1_true,id2_true,label,molfile1,molfile2,id1_c,id2_c,pair_key,id1_pred,id2_pred,...,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag
31,CHEBI_27372,CHEBI_27374,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_27372,CHEBI_27374,"(CHEBI_27372, CHEBI_27374)",CHEBI_27374,CHEBI_27372,...,233.0,"[""chebi:CHEBI:27374""]","[""chebi:CHEBI:27372""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
46,CHEBI_21397,CHEBI_21100,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_21397,CHEBI_21100,"(CHEBI_21100, CHEBI_21397)",CHEBI_21100,CHEBI_21397,...,417.0,"[""chebi:CHEBI:21100""]","[""chebi:CHEBI:21397""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
47,CHEBI_21101,CHEBI_21398,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_21101,CHEBI_21398,"(CHEBI_21101, CHEBI_21398)",CHEBI_21101,CHEBI_21398,...,409.0,"[""chebi:CHEBI:21101""]","[""chebi:CHEBI:21398""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
49,CHEBI_47537,CHEBI_16002,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_47537,CHEBI_16002,"(CHEBI_16002, CHEBI_47537)",CHEBI_16002,CHEBI_47537,...,404.0,"[""chebi:CHEBI:16002""]","[""chebi:CHEBI:47537""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
110,CHEBI_18481,CHEBI_15387,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_18481,CHEBI_15387,"(CHEBI_15387, CHEBI_18481)",CHEBI_18481,CHEBI_15387,...,1153.0,"[""chebi:CHEBI:18481""]","[""chebi:CHEBI:15387""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
164,CHEBI_28548,CHEBI_37209,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_28548,CHEBI_37209,"(CHEBI_28548, CHEBI_37209)",CHEBI_28548,CHEBI_37209,...,401.0,"[""chebi:CHEBI:28548""]","[""chebi:CHEBI:37209""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
175,CHEBI_37547,CHEBI_37546,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_37547,CHEBI_37546,"(CHEBI_37546, CHEBI_37547)",CHEBI_37546,CHEBI_37547,...,416.0,"[""chebi:CHEBI:37546""]","[""chebi:CHEBI:37547""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
232,CHEBI_38200,CHEBI_27669,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_38200,CHEBI_27669,"(CHEBI_27669, CHEBI_38200)",CHEBI_27669,CHEBI_38200,...,2452.0,"[""chebi:CHEBI:27669""]","[""chebi:CHEBI:38200""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
266,CHEBI_17521,CHEBI_36124,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_17521,CHEBI_36124,"(CHEBI_17521, CHEBI_36124)",CHEBI_17521,CHEBI_36124,...,2.0,"[""chebi:CHEBI:17521""]","[""chebi:CHEBI:36124""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0
278,CHEBI_46995,CHEBI_46987,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_46995,CHEBI_46987,"(CHEBI_46987, CHEBI_46995)",CHEBI_46987,CHEBI_46995,...,1781.0,"[""chebi:CHEBI:46987""]","[""chebi:CHEBI:46995""]",1.0,1.0,Diastereomers,100.0,"{""confidence_bin"":""high""}",None,v1.0


Lets investigate these cases manually to determine what went wrong.

Upon manual review, it has been determined that 23 out of the 37 pairs are deemed to be actual correct predictions by the stereomapper pipeline. An the 23 cases, each has been identified to be a mistake in assignment by ChEBI. Most of these relationships are indeed diastereomers, whilst others contain missing stereochemistry, which is picked up by the stereomapper pipeline. In one case, two structures are indeed enantiomers, but the two structures actually share different formal charges meaning they are not deemed enantiomers by the pipeline.

In reality, when accounting for these disagreements, stereomapper has the actual precision, recall and F1 score accounted for below:

In [75]:
df_fp_parent_child = df_fp[df_fp['classification'] == "Parent-child"]
df_fp_parent_child

,id1_true,id2_true,label,molfile1,molfile2,id1_c,id2_c,pair_key,id1_pred,id2_pred,...,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag
65,CHEBI_32922,CHEBI_35440,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_32922,CHEBI_35440,"(CHEBI_32922, CHEBI_35440)",CHEBI_32922,CHEBI_35440,...,1870.0,"[""chebi:CHEBI:32922""]","[""chebi:CHEBI:35440""]",1.0,1.0,Parent-child,79.0,"{""confidence_bin"":""medium""}",None,v1.0
172,CHEBI_37477,CHEBI_37476,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_37477,CHEBI_37476,"(CHEBI_37476, CHEBI_37477)",CHEBI_37477,CHEBI_37476,...,549.0,"[""chebi:CHEBI:37477""]","[""chebi:CHEBI:37476""]",1.0,1.0,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
210,CHEBI_38153,CHEBI_38154,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_38153,CHEBI_38154,"(CHEBI_38153, CHEBI_38154)",CHEBI_38154,CHEBI_38153,...,1579.0,"[""chebi:CHEBI:38154""]","[""chebi:CHEBI:38153""]",1.0,1.0,Parent-child,76.0,"{""confidence_bin"":""medium""}",None,v1.0
237,CHEBI_3332,CHEBI_38969,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_3332,CHEBI_38969,"(CHEBI_3332, CHEBI_38969)",CHEBI_38969,CHEBI_3332,...,622.0,"[""chebi:CHEBI:38969""]","[""chebi:CHEBI:3332""]",1.0,1.0,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0
254,CHEBI_39313,CHEBI_39309,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_39313,CHEBI_39309,"(CHEBI_39309, CHEBI_39313)",CHEBI_39309,CHEBI_39313,...,1570.0,"[""chebi:CHEBI:39309""]","[""chebi:CHEBI:39313""]",1.0,1.0,Parent-child,76.0,"{""confidence_bin"":""medium""}",None,v1.0
256,CHEBI_39335,CHEBI_39336,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_39335,CHEBI_39336,"(CHEBI_39335, CHEBI_39336)",CHEBI_39336,CHEBI_39335,...,932.0,"[""chebi:CHEBI:39336""]","[""chebi:CHEBI:39335""]",1.0,1.0,Parent-child,76.0,"{""confidence_bin"":""medium""}",None,v1.0
588,CHEBI_63698,CHEBI_63695,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_63698,CHEBI_63695,"(CHEBI_63695, CHEBI_63698)",CHEBI_63698,CHEBI_63695,...,1373.0,"[""chebi:CHEBI:63698""]","[""chebi:CHEBI:63695""]",1.0,1.0,Parent-child,77.0,"{""confidence_bin"":""medium""}",None,v1.0
589,CHEBI_63705,CHEBI_63700,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_63705,CHEBI_63700,"(CHEBI_63700, CHEBI_63705)",CHEBI_63705,CHEBI_63700,...,1372.0,"[""chebi:CHEBI:63705""]","[""chebi:CHEBI:63700""]",1.0,1.0,Parent-child,77.0,"{""confidence_bin"":""medium""}",None,v1.0
705,CHEBI_77604,CHEBI_77599,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_77604,CHEBI_77599,"(CHEBI_77599, CHEBI_77604)",CHEBI_77599,CHEBI_77604,...,1788.0,"[""chebi:CHEBI:77599""]","[""chebi:CHEBI:77604""]",1.0,1.0,Parent-child,94.0,"{""confidence_bin"":""high""}",None,v1.0
777,CHEBI_83301,CHEBI_83300,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_83301,CHEBI_83300,"(CHEBI_83300, CHEBI_83301)",CHEBI_83300,CHEBI_83301,...,1708.0,"[""chebi:CHEBI:83300""]","[""chebi:CHEBI:83301""]",1.0,1.0,Parent-child,100.0,"{""confidence_bin"":""high""}",None,v1.0


All are incorrect predictions by StereoMapper.

In [76]:
df_fp_unresolved = df_fp[df_fp['classification'] == "Unresolved"]
df_fp_unresolved

,id1_true,id2_true,label,molfile1,molfile2,id1_c,id2_c,pair_key,id1_pred,id2_pred,...,cluster_b,cluster_a_members,cluster_b_members,cluster_a_size,cluster_b_size,classification,score,score_details,extra_info,version_tag
168,CHEBI_15386,CHEBI_37465,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_15386,CHEBI_37465,"(CHEBI_15386, CHEBI_37465)",CHEBI_37465,CHEBI_15386,...,503.0,"[""chebi:CHEBI:37465""]","[""chebi:CHEBI:15386""]",1.0,1.0,Unresolved,100.0,"{""confidence_bin"":""high""}",Possible pipeline error - should be no identic...,v1.0
208,CHEBI_38139,CHEBI_44343,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_38139,CHEBI_44343,"(CHEBI_38139, CHEBI_44343)",CHEBI_38139,CHEBI_44343,...,1566.0,"[""chebi:CHEBI:38139""]","[""chebi:CHEBI:44343""]",1.0,1.0,Unresolved,100.0,"{""confidence_bin"":""high""}",Possible pipeline error - should be no identic...,v1.0
281,CHEBI_47011,CHEBI_47008,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_47011,CHEBI_47008,"(CHEBI_47008, CHEBI_47011)",CHEBI_47008,CHEBI_47011,...,1780.0,"[""chebi:CHEBI:47008""]","[""chebi:CHEBI:47011""]",1.0,1.0,Unresolved,100.0,"{""confidence_bin"":""high""}",Possible pipeline error - should be no identic...,v1.0
763,CHEBI_83129,CHEBI_83131,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_83129,CHEBI_83131,"(CHEBI_83129, CHEBI_83131)",CHEBI_83131,CHEBI_83129,...,2221.0,"[""chebi:CHEBI:83131""]","[""chebi:CHEBI:83129""]",1.0,1.0,Unresolved,75.0,"{""confidence_bin"":""medium""}",Possible pipeline error - should be no identic...,v1.0
764,CHEBI_83132,CHEBI_83130,Enantiomers,/media/JACK/stereomapper_structures/chebi_struc...,/media/JACK/stereomapper_structures/chebi_struc...,CHEBI_83132,CHEBI_83130,"(CHEBI_83130, CHEBI_83132)",CHEBI_83132,CHEBI_83130,...,2219.0,"[""chebi:CHEBI:83132""]","[""chebi:CHEBI:83130""]",1.0,1.0,Unresolved,75.0,"{""confidence_bin"":""medium""}",Possible pipeline error - should be no identic...,v1.0


all incorrect by StereoMapper. the single unclassified case is a complex case where stereochemistry differs, but so does protonation states.

In [78]:
# 23 for corrected FP in diastereomers, 1 for corrected FP in unclassified
tp_act = tp + 24
fp_act = fp - 24

actual_precision = tp_act / (tp_act + fp_act) if (tp_act + fp_act) > 0 else 0
actual_recall = tp_act / (tp_act + fn) if (tp_act + fn) > 0 else 0
actual_f1 = 2 * actual_precision * actual_recall / (actual_precision + actual_recall) if (actual_precision + actual_recall) > 0 else 0

print(f"Adjusted Precision: {actual_precision:.3f}")
print(f"Adjusted Recall:    {actual_recall:.3f}")
print(f"Adjusted F1 score:  {actual_f1:.3f}")

Adjusted Precision: 0.969
Adjusted Recall:    0.993
Adjusted F1 score:  0.980
